In [1]:
from cmath import sqrt
from cmath import pi
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tnrange, tqdm_notebook
CONVERGENCE_EPS = 1e-7
DIF_EPS = 1e-1
PLOT_STEP = 5 * 1e-4
STOP_IND = 20
def newton_method(start: complex, need_iteration_sequence = False):
    iteration_sequence = [start]
    change = 1
    prev: complex
    next = start
    ind = 1
    while change >= CONVERGENCE_EPS and ind < STOP_IND:
        prev = next
        next = prev - ((prev ** 3 - (1 + 0j)) / (3 * prev ** 2))
        change = abs(next - prev)
        if need_iteration_sequence:
            iteration_sequence.append(next)
        ind += 1
    return next, np.array(iteration_sequence)

def calc_root_pools():
    pools = [[] for _ in range(4)] 
    firs_root = 1 + 0j
    second_root = complex(-0.5, (sqrt(3)/2))
    third_root = complex(-0.5, -(sqrt(3)/2))
    line = np.arange(-2, 2, step = PLOT_STEP)
    for real in tqdm_notebook(line, desc = '1st'):
        for image in line:
            root, _ = newton_method(complex(real, image))
            distances_to_roots = [abs(root - firs_root), abs(root - second_root), abs(root - third_root)]
            ind_root = np.argmin(distances_to_roots)
            #consider point didn't convergence to correct root if distance to each of it more then DIF_EPS (0.1 in current modification)
            if min(distances_to_roots) < DIF_EPS:
                pools[ind_root].append((real, image))
            else:
                pools[3].append((real, image))

    return np.array(pools[0], dtype=tuple),\
           np.array(pools[1], dtype=tuple),\
           np.array(pools[2], dtype=tuple),\
           np.array(pools[3], dtype=tuple)       

first_root_pool, second_root_pool,third_root_pool, uncorrected_root_pool = calc_root_pools()

In [2]:
data = (np.transpose(first_root_pool), np.transpose(second_root_pool), np.transpose(third_root_pool), np.transpose(uncorrected_root_pool))            
colors = ("red", "green", "blue", "black")
groups = ("1", "-0.5 + sqrt(3)/2i", "-0.5 - sqrt(3)/2i", "uncorrected_plots")

# Create plot
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for point, color, group in zip(data, colors, groups):
    if point.size >= 2:
        x, y = point
        ax.scatter(x, y, alpha=0.8, c=color, edgecolors='none', s=30, label=group)
    
plt.title('Newton\'s method for equation z^3 - 1 = 0')
plt.legend(loc=3)
plt.show()
    